# Dies ist der Code um die Daten von unserer Sensebox in die Datenbank zu exportieren

Wir haben uns dazu entschieden, dass die Speicherung der Daten direkt in der Datenbank passiert und keine Zwischenspeicherung als CSV notwendig ist.

In [70]:
import os
import json
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime
import requests

In [71]:
# Verbindungsparameter zur PostgreSQL-Datenbank
db_params = {
    'dbname': 'Wetterstation',
    'user': 'postgres',
    'password': 'Montag1618',
    'host': 'localhost',
    'port': '5432'
}

# Verbindung zur PostgreSQL-Datenbank herstellen
conn = psycopg2.connect(**db_params)

# Erstellen der Tabelle, falls sie noch nicht existiert
create_table_query = """
CREATE TABLE IF NOT EXISTS Wetterstation (
    id SERIAL PRIMARY KEY,
    location_lat DOUBLE PRECISION,
    location_lon DOUBLE PRECISION,
    created_at TIMESTAMP,
    value DOUBLE PRECISION,
    folder_name VARCHAR(255),
    sensor_id VARCHAR(50),
    outlier BOOLEAN
);
"""
with conn.cursor() as cur:
    cur.execute(create_table_query)
    conn.commit()

# Base URL OpenSenseMap API
base_url = "https://api.opensensemap.org/boxes"
senseBoxId = "6645e1c8eb5aad0007b226b5"

# Liste der Sensoren
sensorIds = [
    "6645e1c8eb5aad0007b226b9",  
    "6645e1c8eb5aad0007b226b7",
    "6645e1c8eb5aad0007b226b6",
    "6645e1c8eb5aad0007b226b8"
]

# Parameter
params = {
    'format': 'json',            
    'from-date': '2024-05-17T19:00:00Z',
    'download': 'true',          
    'outliers': 'mark',          
    'outlier-window': 15,        
    'delimiter': 'comma'         
}

all_data = []

for sensorId in sensorIds:
    endpoint = f'{base_url}/{senseBoxId}/data/{sensorId}'
    response = requests.get(endpoint, params=params)
    
    # ob die request erfolgreich war
    if response.status_code == 200:
        data = response.json()
        # sensor ID zu jeder messung hinzufügen 
        for measurement in data:
            measurement['sensor_id'] = sensorId
            # nach outliers schauen und 'outlier' hinzufügen
            if 'outlier' in measurement:
                measurement['outlier'] = measurement['outlier']
            else:
                measurement['outlier'] = None  # falls keine outlier, setzen auf None
        all_data.extend(data)
    else:
        print(f'Error retrieving data for sensor {sensorId}: {response.status_code}')

# Daten in die PostgreSQL-Datenbank einfügen
if all_data:
    # Umwandeln der Daten in das passende Format für execute_values
    formatted_data = [
        (
            entry.get('location', [None, None])[0],  # location_lat
            entry.get('location', [None, None])[1],  # location_lon
            datetime.strptime(entry.get('createdAt'), '%Y-%m-%dT%H:%M:%S.%fZ'),                  # created_at
            float(entry.get('value', 0)),            # value
            None,                                    # folder_name (nicht vorhanden in API-Daten)
            entry.get('sensor_id'),                  # sensor_id
            entry.get('outlier') is not None         # outlier
        )
        for entry in all_data
    ]
    
    insert_query = """
    INSERT INTO Wetterstation (location_lat, location_lon, created_at, value, folder_name, sensor_id, outlier)
    VALUES %s
    """
    with conn.cursor() as cur:
        execute_values(cur, insert_query, formatted_data)
        conn.commit()
    print("Daten erfolgreich in die Datenbank eingefügt.")
else:
    print("Keine Daten abgerufen oder Liste ist leer.")

# Verbindung schließen
conn.close()

Daten erfolgreich in die Datenbank eingefügt.
